## NLP Sentiment Analysis

In [61]:
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

In [62]:
data = pd.read_csv('train.tsv', sep='\t', header=0)

### each row in our data has these features:
    - unique PhraseId
    - original sentenceId
    - Pharse
    - label- sentiment 

sentiment is from 0 - very negative to 4- very positive

In [63]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [64]:
#tokenize each pharse to list of words

In [65]:
from tokenizing import Tokenizer

In [66]:
tok = Tokenizer(preserve_case=True)
data['Phrase'] = data['Phrase'].apply(tok.tokenize)


In [67]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"[A, series, of, escapades, demonstrating, the,...",1
1,2,1,"[A, series, of, escapades, demonstrating, the,...",2
2,3,1,"[A, series]",2
3,4,1,[A],2
4,5,1,[series],2


In [68]:
# # Remove stop words from phrases

# import nltk

# from nltk.corpus import stopwords # Import the stop word list
# print stopwords.words("english") 



In [69]:
# data['Phrase'] = data['Phrase'].apply(lambda words:[w for w in words if not w in stopwords.words("english")])

In [70]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"[A, series, of, escapades, demonstrating, the,...",1
1,2,1,"[A, series, of, escapades, demonstrating, the,...",2
2,3,1,"[A, series]",2
3,4,1,[A],2
4,5,1,[series],2


###  Naive Classifier

In [71]:
# use avg sentiment of the words in the pharse to estimate pharse sentiment
# train on ~90% of the data and check on the rest

In [72]:
sentiment_dict = {}
max_train = 140353
for i,word_list in enumerate(data['Phrase'][:max_train]):
    if len(word_list)==1:
        sentiment_dict[word_list[0].lower()] = data.Sentiment[i]
    elif len(word_list)==0:
        data.drop(data.index[[i]],inplace=True)


In [73]:
def avg_sentiment(word_list):
    sen_sum =0
    for word in word_list:
        sen_sum += sentiment_dict.get(word.lower(),2)
    return sen_sum/len(word_list)

data['avg sentiment'] = data['Phrase'][max_train:].apply(avg_sentiment)


### check results of classifier

In [74]:
predict = data['avg sentiment'][max_train:]
labels = data['Sentiment'][max_train:]

In [75]:
print classification_report(labels,predict)

             precision    recall  f1-score   support

          0       1.00      0.00      0.01       746
          1       0.41      0.41      0.41      3142
          2       0.52      0.84      0.65      7668
          3       0.51      0.03      0.06      3239
          4       0.00      0.00      0.00       911

avg / total       0.49      0.50      0.41     15706



# bag of words

In [100]:
data.shape[0]



156059

In [95]:
#creating vocabulary
NUM_OF_PHARSE = data.shape[0]
vocab = []
for i in range(NUM_OF_PHARSE):
    vocab.extend(data['Phrase'].iloc[i])
vocab = list(set(vocab))


In [96]:
len(vocab)

18097

In [97]:
data['Bag_of_Words']=pd.Series(0)
data.head()

,PhraseId,SentenceId,Phrase,Sentiment,avg sentiment,Bag_of_Words
0,1,1,"[A, series, of, escapades, demonstrating, the,...",1,NaN,0.0
1,2,1,"[A, series, of, escapades, demonstrating, the,...",2,NaN,NaN
2,3,1,"[A, series]",2,NaN,NaN
3,4,1,[A],2,NaN,NaN
4,5,1,[series],2,NaN,NaN


In [98]:
bag_of_words = np.zeros((NUM_OF_PHARSE,len(vocab)))
for i in range(NUM_OF_PHARSE):
    for word in data['Phrase'][i]:
        indx = vocab.index(word)
        bag_of_words[i][indx] +=1

MemoryError: 

In [99]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 5) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( bag_of_words, data["Sentiment"] )

Training the random forest...


ValueError: Number of labels=156059 does not match number of samples=15390